# Deformable Attention 模型结构
## 一、模型简介
DeformableAttention 是可变形注意力机制的 1 维单头实现，核心特点是通过学习偏移量动态调整采样位置，相比传统注意力更灵活。主要用于从输入特征（Value）中根据查询（Query）动态采样关键信息并加权融合。
## 二、核心参数与组件
组件	                 作用	                                      关键参数
sampling_offsets	线性层，从 Query 估计采样点偏移量  输入维度：hidden_size，输出维度：n_points * 1 * 2（单头 2D 偏移）
attention_weights	线性层，从 Query 估计采样点权重	 输入维度：hidden_size，输出维度：n_points * 1（单头权重）
## 三、Shape 变化详解（以示例输入为例）
示例输入参数：
batch_size=1，sequence_length=8，hidden_size=16，n_points=3
1. 输入参数 Shape
参数	           形状	                   说明
q（Query）	(1, 8, 16)	(batch_size, seq_len, hidden_size)
v（Value）	(1, 8, 16)	待采样的特征向量
reference_points	(1, 8, 2)	初始参考点（归一化到 [0,1]）
input_spatial_shapes	(2,) → [8, 1]	输入特征的空间尺寸（长度 = 8，高度 = 1）
2. 偏移量与权重估计
方法：esitimate_offset_and_weights 或 ground_truth_offset_and_weights
输出 Shape：
sampling_offsets：(1, 8, 3, 2)
（batch_size, seq_len, n_points, 2，每个采样点的 2D 偏移量）
weights：(1, 8, 3)
（batch_size, seq_len, n_points，经 softmax 归一化的权重）
3. 采样点位置计算
参考点增维 + 偏移量归一化
sampling_locations = reference_points[:, :, None, :] + sampling_offsets / offset_normalizer[None, None, None, :]
reference_points 增维：(1,8,2) → (1,8,1,2)（新增采样点维度）
offset_normalizer 增维：(2,) → (1,1,1,2)（广播适配）
sampling_locations 形状：(1, 8, 3, 2)（归一化坐标，范围 [0,1] 附近）
转换为绝对坐标
sampling_locations_absolute = sampling_locations * input_spatial_shapes[None, None, None, :]
形状不变：(1, 8, 3, 2)（映射为实际序列索引，如 0~8）
4. 特征采样（F.grid_sample）
Value 维度调整
v = v.permute(0, 2, 1).unsqueeze(-1)  # (1,8,16) → (1,16,8) → (1,16,8,1)
适配 grid_sample 输入格式：(batch, channel, height, width)（这里 height=1，width=8）
网格采样
sampled_values = F.grid_sample(v, sampling_locations, align_corners=True)
输出形状：(1, 16, 8, 3)
（batch_size, hidden_size, seq_len, n_points，每个采样点提取的特征）
5. 加权求和输出
维度调整
sampled_values = sampled_values.permute(0, 2, 3, 1)  # (1,16,8,3) → (1,8,3,16)
加权求和
output = torch.sum(sampled_values * weights.unsqueeze(-1), dim=-2)
weights 增维：(1,8,3) → (1,8,3,1)
输出形状：(1, 8, 16)（batch_size, seq_len, hidden_size，与输入特征维度一致）
## 四、输出结果 Shape
输出	       形状	        说明
output	(1, 8, 16)	可变形注意力加权后的特征向量
sampling_locations_absolute	(1, 8, 3, 2)	采样点的绝对坐标
deformable_score（权重）	(1, 8, 3)	每个采样点的注意力权重
## 五、核心流程总结
从 Query 估计 / 获取偏移量和权重 → 确定动态采样点
结合参考点计算采样点坐标 → 映射到实际序列位置
从 Value 中采样特征 → 按权重加权求和 → 输出最终特征
通过动态调整采样位置，模型能更灵活地聚焦关键信息，适用于目标检测、序列建模等任务。